PyTorch Basics

In [5]:
# PyTorch Basics
import torch
import torch.nn as nn
import torch.optim as optim

#Tensors
print("Tensors:")
x = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
y = torch.tensor([[2.0], [1.0]])
print("x:", x)
print("y:", y)

#Tensors
print("\nAutograd:")
z = (x @ y) ** 2   # matrix multiply then square
loss = z.sum()
print("Loss:", loss.item())

loss.backward()
print("Gradients of x:\n", x.grad)

#Simple Training Loop
print("\nSimple Training Loop (Linear Regression)")

# Training data: y = 3x + noise
torch.manual_seed(0)
X = torch.linspace(0, 1, 20).unsqueeze(1)
Y = 3 * X + 0.2 * torch.randn_like(X)

model = nn.Linear(1, 1)  # single-layer linear model
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = criterion(y_pred, Y)
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

print("Trained weight, bias:", list(model.parameters()))


Tensors:
x: tensor([[1., 2.],
        [3., 4.]], requires_grad=True)
y: tensor([[2.],
        [1.]])

Autograd:
Loss: 116.0
Gradients of x:
 tensor([[16.,  8.],
        [40., 20.]])

Simple Training Loop (Linear Regression)
Epoch 0: Loss = 2.4849
Epoch 20: Loss = 0.6642
Epoch 40: Loss = 0.3832
Epoch 60: Loss = 0.2267
Epoch 80: Loss = 0.1395
Trained weight, bias: [Parameter containing:
tensor([[2.5781]], requires_grad=True), Parameter containing:
tensor([0.3120], requires_grad=True)]


PINNs Example- 
We’ll solve a 1D heat equation, ut = αuxx, x∈[0,1],t∈[0,1]
with α = 0.1, and initial condition u(x,0)=sin(πx).

In [6]:
# PINN for 1D Heat Equation
import torch
import torch.nn as nn
import torch.optim as optim

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Neural Networks 
class MLP(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers)-1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))
    
    def forward(self, x):
        for i, l in enumerate(self.layers[:-1]):
            x = torch.tanh(l(x))
        return self.layers[-1](x)

# Problem setup 
alpha = 0.1

# Sample collocation points (x,t)
N_f = 1000
x_f = torch.rand(N_f,1)
t_f = torch.rand(N_f,1)
X_f = torch.cat([x_f, t_f], dim=1).to(device)

# Initial condition: u(x,0) = sin(pi x)
N_ic = 100
x_ic = torch.linspace(0,1,N_ic).unsqueeze(1)
t_ic = torch.zeros_like(x_ic)
u_ic = torch.sin(torch.pi * x_ic)
X_ic = torch.cat([x_ic, t_ic], dim=1).to(device)
u_ic = u_ic.to(device)

# Model
model = MLP([2, 32, 32, 32, 1]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Loss functions
def pde_residual(X):
    """Heat equation residual: u_t - alpha u_xx"""
    X.requires_grad_(True)
    u = model(X)
    grads = torch.autograd.grad(u, X, 
                                grad_outputs=torch.ones_like(u),
                                create_graph=True)[0]
    u_x = grads[:,0:1]
    u_t = grads[:,1:2]
    
    u_xx = torch.autograd.grad(u_x, X, 
                               grad_outputs=torch.ones_like(u_x),
                               create_graph=True)[0][:,0:1]
    
    return u_t - alpha * u_xx

def loss_fn():
    # PDE loss
    f = pde_residual(X_f)
    loss_f = (f**2).mean()
    # IC loss
    u_pred = model(X_ic)
    loss_ic = ((u_pred - u_ic)**2).mean()
    return loss_f + loss_ic

# Training
for epoch in range(2000):
    optimizer.zero_grad()
    loss = loss_fn()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")


Epoch 0: Loss = 0.328581
Epoch 200: Loss = 0.078552
Epoch 400: Loss = 0.000552
Epoch 600: Loss = 0.000075
Epoch 800: Loss = 0.000042
Epoch 1000: Loss = 0.000027
Epoch 1200: Loss = 0.000019
Epoch 1400: Loss = 0.000013
Epoch 1600: Loss = 0.000035
Epoch 1800: Loss = 0.000138
